In [1]:
import pandas as pd
import numpy as np
import nltk

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [3]:
train = pd.read_csv('ta-en-misogyny-train.csv',sep='\t', names = ["category", "text"])
train = train.dropna()

In [ ]:
train

,category,text
0,None-of-the-above,enaku unmaikum aluha wantu thirunangaigal thei...
1,Transphobic,SUPERSTAR VIJAY dai arivuketta polu ithu thapp...
2,None-of-the-above,Ugka smile cute a iruku😊😊...
3,None-of-the-above,Anna i am waiting na 🥰🥰🥰
4,None-of-the-above,Yanda tamilnadu la evvalavo pirachana iruku at...
...,...,...
5943,None-of-the-above,Noq day having no caste no religious certificate
5944,None-of-the-above,Mimicry escape aitaru bro. 😂
5945,None-of-the-above,Rajesh R u age u from
5946,Counter-speech,Bro unga videos nallarukku but don't do prank ...


In [4]:
labels = train.category.unique()
labels

array(['None-of-the-above', 'Transphobic', 'Counter-speech', 'Misandry',
       'Homophobia', 'Hope-Speech', 'Xenophobia', 'Misogyny'],
      dtype=object)

In [5]:
train = pd.read_csv('ta-en-misogyny-train.csv',sep='\t', names = ["category", "text"])
train = train.dropna()

In [7]:
test = pd.read_csv('ta-en-misogyny-dev.csv',sep='\t', names = ["category", "text"])
test = test.dropna()

In [9]:
import string
def removepunctuation(text):
    clean="".join([i for i in text if i not in string.punctuation])
    return clean

import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def tolower(text):
  return text.lower()

label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['la', 'u','r','s','bro','da','dei','dai','nu','ah','nee','ni','illa','un','ok','na','pls','ur','unga']) 
stopwords_list.extend(['enna','va','hi','h','ku','iruku','naa','va','oru','athu','avanga','neenga','tha','en','di','dhan','ne','ella','intha']) 

label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

def removestopwords(text):
  clean = " ".join([word for word in text.split() if word not in stopwords_list])
  return clean

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
def preprocess(df):
  df['text']= df['text'].apply(lambda x:tolower(x))
  df['text']= df['text'].apply(lambda x:removepunctuation(x))
  df['text']= df['text'].apply(lambda x:deEmojify(x))
  df['text']= df['text'].apply(lambda x:removestopwords(x))
  df["category"] = df["category"].map(label)
  return df

In [12]:
train = preprocess(train)
train

,category,text
0,0,enaku unmaikum aluha wantu thirunangaigal thei...
1,1,superstar vijay arivuketta polu ithu thappu pa...
2,0,ugka smile cute
3,0,anna waiting 🥰🥰🥰
4,0,yanda tamilnadu evvalavo pirachana athalam vit...
...,...,...
5943,0,noq day caste religious certificate
5944,0,mimicry escape aitaru
5945,0,rajesh age
5946,2,videos nallarukku dont prank eena varavanga ep...


In [13]:
test = preprocess(test)

In [14]:
def tokenize(text):
  return text.split()


train['token']= train['text'].apply(lambda x:tokenize(x))
test['token']= test['text'].apply(lambda x:tokenize(x))

In [15]:
train

,category,text,token
0,0,enaku unmaikum aluha wantu thirunangaigal thei...,"[enaku, unmaikum, aluha, wantu, thirunangaigal..."
1,1,superstar vijay arivuketta polu ithu thappu pa...,"[superstar, vijay, arivuketta, polu, ithu, tha..."
2,0,ugka smile cute,"[ugka, smile, cute]"
3,0,anna waiting 🥰🥰🥰,"[anna, waiting, 🥰🥰🥰]"
4,0,yanda tamilnadu evvalavo pirachana athalam vit...,"[yanda, tamilnadu, evvalavo, pirachana, athala..."
...,...,...,...
5943,0,noq day caste religious certificate,"[noq, day, caste, religious, certificate]"
5944,0,mimicry escape aitaru,"[mimicry, escape, aitaru]"
5945,0,rajesh age,"[rajesh, age]"
5946,2,videos nallarukku dont prank eena varavanga ep...,"[videos, nallarukku, dont, prank, eena, varava..."


In [16]:
from collections import Counter
x = train.groupby("category")["text"].apply(lambda x: Counter(" ".join(x).split()).most_common(100))
x[6]

[('china', 120),
 ('virus', 19),
 ('tamil', 17),
 ('corona', 14),
 ('jaadhi', 14),
 ('indha', 13),
 ('sappa', 13),
 ('thulukka', 13),
 ('thayvidiyaa', 13),
 ('avan', 12),
 ('india', 11),
 ('world', 11),
 ('first', 11),
 ('antha', 11),
 ('vera', 10),
 ('mela', 10),
 ('iruka', 10),
 ('peru', 10),
 ('karan', 10),
 ('irundhu', 10),
 ('innum', 9),
 ('america', 8),
 ('pottu', 8),
 ('chinna', 8),
 ('vittu', 8),
 ('ulaga', 8),
 ('nalla', 8),
 ('kooti', 8),
 ('pola', 8),
 ('bomb', 8),
 ('sari', 8),
 ('chaina', 7),
 ('ya', 7),
 ('summa', 7),
 ('enga', 7),
 ('thaan', 7),
 ('vida', 7),
 ('vanthu', 7),
 ('ena', 7),
 ('yen', 7),
 ('kooda', 7),
 ('mattum', 7),
 ('ellam', 7),
 ('ithu', 6),
 ('matum', 6),
 ('daa', 6),
 ('dey', 6),
 ('poda', 6),
 ('vandhu', 6),
 ('idhu', 6),
 ('payal', 6),
 ('poi', 6),
 ('panni', 6),
 ('pala', 6),
 ('naayae', 6),
 ('um', 6),
 ('ban', 6),
 ('vendum', 6),
 ('nattai', 6),
 ('latheef', 6),
 ('map', 5),
 ('ellarum', 5),
 ('endha', 5),
 ('chappa', 5),
 ('onnum', 5),
 ('vara',

In [17]:
processed = [key for groups in x for key, value in groups]


In [18]:
uniqueset = set(processed)
processed = list(uniqueset) 

In [19]:
processed

['indha',
 'romba',
 'venum',
 'panni',
 'po',
 'nature',
 'interview',
 'irukanum',
 'pana',
 'speech',
 'thevadiya',
 'nattai',
 'parents',
 'boy',
 'aan',
 'gopi',
 'number',
 'boys',
 'sappa',
 'happy',
 'echa',
 'chappa',
 'semma',
 'mattum',
 'pondati',
 'peru',
 'avan',
 'ivlo',
 'ungal',
 'chinava',
 'unaku',
 'payale',
 'pottu',
 'ava',
 'irundhu',
 'aana',
 'vada',
 'mudiyala',
 'ombodhu',
 'thappa',
 'irukku',
 'thanga',
 'ipo',
 'idhunga',
 'ivanga',
 'ungaluku',
 'storm',
 'avala',
 'sex',
 'aluga',
 'vai',
 'naai',
 'vara',
 'anga',
 'nega',
 'sister',
 'sonna',
 'babu',
 'ponnu',
 'porantha',
 'rape',
 'gay',
 'ungomma',
 'level',
 'naye',
 'virupam',
 'doctor',
 'god',
 'girls',
 'vera',
 'pesura',
 'kaasu',
 'naaye',
 'ippadi',
 'samy',
 'arivu',
 'neega',
 'onnu',
 'ram',
 'sema',
 'mavane',
 'enka',
 'dhevudiya',
 'naya',
 'apo',
 'mannippu',
 'support',
 'pakistan',
 'pola',
 'thambi',
 'muslim',
 'iruka',
 'latheef',
 'pasangala',
 'naadu',
 'summa',
 'solla',
 'ep

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [27]:
count_vect = CountVectorizer(vocabulary=processed, max_df=0.75, min_df=5, max_features=10000)

In [28]:
X_train = count_vect.fit_transform(train["text"])

In [29]:
X_train

<5943x419 sparse matrix of type '<class 'numpy.int64'>'
	with 16351 stored elements in Compressed Sparse Row format>

In [30]:
y_train = train["category"]

In [31]:
from sklearn import svm
svm_model = svm.SVC(kernel='rbf', C=1, verbose=True)
svm_model.fit(X_train, y_train)

[LibSVM]

SVC(C=1, verbose=True)

In [32]:
X_test = count_vect.fit_transform(test["text"])
y_test = test["category"]

In [33]:
y_pred = svm_model.predict(X_test)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.98      0.82       917
           1       0.00      0.00      0.00        40
           2       0.43      0.11      0.17        95
           3       0.75      0.47      0.58       218
           4       0.00      0.00      0.00        43
           5       0.00      0.00      0.00        53
           6       1.00      0.44      0.61        70
           7       1.00      0.12      0.21        50

    accuracy                           0.71      1486
   macro avg       0.48      0.26      0.30      1486
weighted avg       0.65      0.71      0.63      1486



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',vocabulary=processed)
X_train = tfidfvectorizer.fit_transform(train["text"])
X_test = tfidfvectorizer.fit_transform(test["text"])

In [38]:
X_train

<5943x419 sparse matrix of type '<class 'numpy.float64'>'
	with 16351 stored elements in Compressed Sparse Row format>

In [48]:
X_train.toarray()[1].size


419

In [36]:
svm_model = svm.SVC(kernel='rbf', C=1, verbose=True)
svm_model.fit(X_train, y_train)

[LibSVM]

SVC(C=1, verbose=True)

In [37]:
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.97      0.82       917
           1       0.50      0.05      0.09        40
           2       0.33      0.01      0.02        95
           3       0.73      0.55      0.62       218
           4       1.00      0.02      0.05        43
           5       1.00      0.02      0.04        53
           6       0.93      0.53      0.67        70
           7       0.83      0.10      0.18        50

    accuracy                           0.71      1486
   macro avg       0.75      0.28      0.31      1486
weighted avg       0.71      0.71      0.64      1486



In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(200, ))
mlp_clf.fit(X_train, y_train)
y_pred = mlp_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.89      0.82       917
           1       0.35      0.20      0.25        40
           2       0.36      0.20      0.26        95
           3       0.65      0.58      0.62       218
           4       0.45      0.30      0.36        43
           5       0.24      0.17      0.20        53
           6       0.71      0.60      0.65        70
           7       0.62      0.30      0.41        50

    accuracy                           0.70      1486
   macro avg       0.52      0.41      0.45      1486
weighted avg       0.67      0.70      0.68      1486



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.93      0.80       917
           1       0.22      0.15      0.18        40
           2       0.15      0.02      0.04        95
           3       0.59      0.39      0.47       218
           4       0.50      0.07      0.12        43
           5       0.32      0.11      0.17        53
           6       0.74      0.44      0.55        70
           7       0.29      0.08      0.12        50

    accuracy                           0.66      1486
   macro avg       0.44      0.27      0.31      1486
weighted avg       0.60      0.66      0.61      1486



In [ ]:
clf = RandomForestClassifier(n_estimators=1500,verbose=True, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   25.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:    0.9s
[Parallel(n_jobs=2)]: Done 1500 out of 1500 | elapsed:    1.1s finished


              precision    recall  f1-score   support

           0       0.73      0.94      0.82       917
           1       0.54      0.17      0.26        40
           2       0.37      0.07      0.12        95
           3       0.66      0.56      0.61       218
           4       0.60      0.21      0.31        43
           5       0.33      0.08      0.12        53
           6       0.71      0.57      0.63        70
           7       0.62      0.16      0.25        50

    accuracy                           0.71      1486
   macro avg       0.57      0.35      0.39      1486
weighted avg       0.67      0.71      0.66      1486



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GB_classifier = GradientBoostingClassifier(verbose=True)
GB_classifier.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.2149            6.05s
         2           1.1725            5.79s
         3           1.1449            5.65s
         4           1.1239            5.64s
         5           1.1042            5.51s
         6           1.0865            5.53s
         7           1.0715            5.48s
         8           1.0597            5.42s
         9           1.0485            5.37s
        10           1.0362            5.29s
        20           0.9551            4.59s
        30           0.8953            4.04s
        40           0.8471            3.45s
        50           0.8105            2.87s
        60           0.7817            2.30s
        70           0.7530            1.72s
        80           0.7332            1.15s
        90           0.7112            0.57s
       100           0.6907            0.00s


GradientBoostingClassifier(verbose=True)

In [ ]:
y_pred = GB_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.95      0.81       917
           1       0.50      0.10      0.17        40
           2       0.44      0.12      0.18        95
           3       0.74      0.44      0.56       218
           4       0.67      0.19      0.29        43
           5       0.25      0.08      0.12        53
           6       0.84      0.53      0.65        70
           7       0.67      0.20      0.31        50

    accuracy                           0.70      1486
   macro avg       0.60      0.33      0.39      1486
weighted avg       0.68      0.70      0.65      1486



In [ ]:
result = pd.read_csv('ta-en-misogyny-test.csv',sep='\t', names = ["text"])
result = result.dropna()
result

,text
0,ante peddi yedukkure ponnai vacci panam sambat...
1,Yella idangalilum pengalai asinga paduthuvathe...
2,Pannikku porandha paradhesi H Raja... Ponna pu...
3,Ayya white T-shirt! Tamil culture pathi mulusa...
4,Madu vandhi bjb fraudulent lady
...,...
1852,Avanga sonanga pari 8:50 sema
1853,Bro Avanga great bro semma character antha akk...
1854,NAGENDRAN NAGA hii Ur watsapp number
1855,Nengal solvathu sari endral angalum kuda vethi...


In [ ]:
result['text']= result['text'].apply(lambda x:tolower(x))
result['text']= result['text'].apply(lambda x:removepunctuation(x))
result['text']= result['text'].apply(lambda x:deEmojify(x))
result['text']= result['text'].apply(lambda x:removestopwords(x))

In [ ]:
Xf = tfidfvectorizer.fit_transform(result["text"])

In [ ]:
yf = mlp_clf.predict(Xf)

In [ ]:
yf.shape

(1857,)

In [ ]:
key = { 0: 'None-of-the-above', 1:'Transphobic', 2: 'Counter-speech', 3:'Misandry',
       4:'Homophobia', 5:'Hope-Speech', 6:'Xenophobia', 7:'Misogyny'}

In [ ]:
list_of_tuples = list(zip(result["text"].tolist(), yf.tolist()))

In [ ]:
finalresult = pd.DataFrame(list_of_tuples,
                  columns = ['test_id', 'class_label'])

In [ ]:
finalresult

,test_id,class_label
0,ante peddi yedukkure ponnai vacci panam sambat...,None-of-the-above
1,yella idangalilum pengalai asinga paduthuvathe...,Misandry
2,pannikku porandha paradhesi raja ponna pundaiy...,Misandry
3,ayya white tshirt tamil culture pathi mulusa p...,None-of-the-above
4,madu vandhi bjb fraudulent lady,None-of-the-above
...,...,...
1852,sonanga pari 850 sema,None-of-the-above
1853,great semma character antha akka avangalukku v...,None-of-the-above
1854,nagendran naga hii watsapp number,None-of-the-above
1855,nengal solvathu sari endral angalum kuda vethi...,None-of-the-above


In [ ]:
finalresult["class_label"] = finalresult["class_label"].map(key)

In [ ]:
finalresult

,test_id,class_label
0,ante peddi yedukkure ponnai vacci panam sambat...,None-of-the-above
1,yella idangalilum pengalai asinga paduthuvathe...,Misandry
2,pannikku porandha paradhesi raja ponna pundaiy...,Misandry
3,ayya white tshirt tamil culture pathi mulusa p...,None-of-the-above
4,madu vandhi bjb fraudulent lady,None-of-the-above
...,...,...
1852,sonanga pari 850 sema,None-of-the-above
1853,great semma character antha akka avangalukku v...,None-of-the-above
1854,nagendran naga hii watsapp number,None-of-the-above
1855,nengal solvathu sari endral angalum kuda vethi...,None-of-the-above


In [ ]:
finalresult.to_csv("pandas_ta-en_run1.tsv", sep='\t', header = None)